# Null Model Training

This notebook trains ML models on null networks (degree-preserving permutations) to create degree-aware null models.

## Purpose

Train models that predict edge probabilities based **only** on degree structure (no biological signal):
- **Training data**: Permutations 1-20 (null networks)
- **Validation data**: Permutations 21-30 (held-out nulls)
- **Test data**: Hetionet (perm 0) vs validation nulls

## Key Features

- **No data leakage**: Hetionet never seen during training
- **Two models**: Polynomial Logistic Regression (fast) + Random Forest (accurate)
- **Error analysis**: Performance stratified by degree bins
- **All 24 edge types**: Complete null model library

## Workflow

1. Load null edge frequencies (perms 1-20)
2. Train Polynomial LogReg and Random Forest
3. Validate on held-out nulls (perms 21-30)
4. Error analysis by degree
5. Save models and validation results

In [ ]:
# Papermill parameters
training_perm_range = (1, 21)  # Permutations 1-20 for training
validation_perm_range = (21, 31)  # Permutations 21-30 for validation
edge_types_to_process = None  # None = all 24 edge types

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr
import joblib
import warnings
warnings.filterwarnings('ignore')

# Setup paths
repo_dir = Path.cwd()
data_dir = repo_dir / 'data'
results_dir = repo_dir / 'results' / 'null_models'
results_dir.mkdir(parents=True, exist_ok=True)

print(f"Repository directory: {repo_dir}")
print(f"Data directory: {data_dir}")
print(f"Results will be saved to: {results_dir}")

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

## 1. Define Edge Types and Helper Functions

In [ ]:
# All 24 edge types
ALL_EDGE_TYPES = [
    "AdG", "AeG", "AuG", "CbG", "CcSE", "CdG", "CpD", "CrC", "CtD", "CuG",
    "DaG", "DdG", "DlA", "DpS", "DrD", "DuG", "GcG", "GiG", "GpBP", "GpCC",
    "GpMF", "GpPW", "Gr>G", "PCiC"
]

# Use provided edge_types or default to all
if edge_types_to_process is None:
    edge_types_to_process = ALL_EDGE_TYPES

print(f"Processing {len(edge_types_to_process)} edge types")
print(f"Training permutations: {training_perm_range[0]}-{training_perm_range[1]-1}")
print(f"Validation permutations: {validation_perm_range[0]}-{validation_perm_range[1]-1}")

In [ ]:
def load_null_edge_frequencies(edge_type, perm_ids, negative_sample_ratio=0.1):
    """
    Load and aggregate edge frequencies from null permutations.
    
    Parameters:
    -----------
    edge_type : str
        Edge type (e.g., 'CbG')
    perm_ids : list or range
        Permutation IDs to load
    negative_sample_ratio : float
        Ratio of negative samples to positive samples
        
    Returns:
    --------
    pd.DataFrame with columns: source_degree, target_degree, edge_probability
    """
    all_pairs = []
    
    for perm_id in perm_ids:
        edge_file = data_dir / 'permutations' / f'{perm_id:03d}.hetmat' / 'edges' / f'{edge_type}.sparse.npz'
        
        if not edge_file.exists():
            print(f"  Warning: File not found: {edge_file}")
            continue
        
        # Load edge matrix
        matrix = sp.load_npz(str(edge_file))
        source_degrees = np.array(matrix.sum(axis=1)).flatten()
        target_degrees = np.array(matrix.sum(axis=0)).flatten()
        
        # Positive examples (edges)
        for i, j in zip(*matrix.nonzero()):
            all_pairs.append({
                'source_degree': int(source_degrees[i]),
                'target_degree': int(target_degrees[j]),
                'edge_exists': 1
            })
        
        # Negative examples (sample non-edges)
        n_nodes_source, n_nodes_target = matrix.shape
        n_negatives = int(matrix.nnz * negative_sample_ratio)
        
        sampled = 0
        attempts = 0
        max_attempts = n_negatives * 10
        
        while sampled < n_negatives and attempts < max_attempts:
            i = np.random.randint(0, n_nodes_source)
            j = np.random.randint(0, n_nodes_target)
            
            if matrix[i, j] == 0 and source_degrees[i] > 0 and target_degrees[j] > 0:
                all_pairs.append({
                    'source_degree': int(source_degrees[i]),
                    'target_degree': int(target_degrees[j]),
                    'edge_exists': 0
                })
                sampled += 1
            attempts += 1
    
    # Convert to DataFrame and aggregate
    df = pd.DataFrame(all_pairs)
    
    if len(df) == 0:
        return pd.DataFrame(columns=['source_degree', 'target_degree', 'edge_probability'])
    
    # Aggregate by degree pair
    null_freq = df.groupby(['source_degree', 'target_degree']).agg({
        'edge_exists': ['sum', 'count', 'mean']
    }).reset_index()
    
    null_freq.columns = ['source_degree', 'target_degree', 'edge_count', 'total_count', 'edge_probability']
    
    return null_freq

In [ ]:
def analyze_errors_by_degree(predictions, actuals, source_degrees, target_degrees):
    """
    Analyze prediction errors stratified by degree bins.
    """
    # Define degree bins
    degree_bins = [0, 1, 2, 5, 10, 20, 50, 100, 500, np.inf]
    degree_labels = ['0', '1', '2-4', '5-9', '10-19', '20-49', '50-99', '100-499', '500+']
    
    results = []
    
    # Bin degrees
    source_bins = pd.cut(source_degrees, bins=degree_bins, labels=degree_labels)
    target_bins = pd.cut(target_degrees, bins=degree_bins, labels=degree_labels)
    
    # Error metrics by source degree
    for bin_label in degree_labels:
        mask = source_bins == bin_label
        if mask.sum() >= 10:
            results.append({
                'stratification': 'source_degree',
                'bin': bin_label,
                'n_samples': int(mask.sum()),
                'mean_prediction': float(predictions[mask].mean()),
                'mean_actual': float(actuals[mask].mean()),
                'mae': float(np.abs(predictions[mask] - actuals[mask]).mean()),
                'rmse': float(np.sqrt(((predictions[mask] - actuals[mask])**2).mean())),
                'correlation': float(np.corrcoef(predictions[mask], actuals[mask])[0,1]) if mask.sum() > 1 else np.nan,
                'mean_degree': float(source_degrees[mask].mean())
            })
    
    # Error metrics by target degree
    for bin_label in degree_labels:
        mask = target_bins == bin_label
        if mask.sum() >= 10:
            results.append({
                'stratification': 'target_degree',
                'bin': bin_label,
                'n_samples': int(mask.sum()),
                'mean_prediction': float(predictions[mask].mean()),
                'mean_actual': float(actuals[mask].mean()),
                'mae': float(np.abs(predictions[mask] - actuals[mask]).mean()),
                'rmse': float(np.sqrt(((predictions[mask] - actuals[mask])**2).mean())),
                'correlation': float(np.corrcoef(predictions[mask], actuals[mask])[0,1]) if mask.sum() > 1 else np.nan,
                'mean_degree': float(target_degrees[mask].mean())
            })
    
    # Error metrics by degree product
    degree_products = source_degrees * target_degrees
    product_bins = pd.cut(degree_products, 
                          bins=[0, 1, 10, 100, 1000, 10000, 100000, np.inf],
                          labels=['0-1', '2-10', '11-100', '101-1K', '1K-10K', '10K-100K', '100K+'])
    
    for bin_label in product_bins.unique():
        if pd.notna(bin_label):
            mask = product_bins == bin_label
            if mask.sum() >= 10:
                results.append({
                    'stratification': 'degree_product',
                    'bin': str(bin_label),
                    'n_samples': int(mask.sum()),
                    'mean_prediction': float(predictions[mask].mean()),
                    'mean_actual': float(actuals[mask].mean()),
                    'mae': float(np.abs(predictions[mask] - actuals[mask]).mean()),
                    'rmse': float(np.sqrt(((predictions[mask] - actuals[mask])**2).mean())),
                    'correlation': float(np.corrcoef(predictions[mask], actuals[mask])[0,1]) if mask.sum() > 1 else np.nan,
                    'mean_product': float(degree_products[mask].mean())
                })
    
    return pd.DataFrame(results)

## 2. Train Null Models for All Edge Types

In [ ]:
training_results = []

for edge_type in edge_types_to_process:
    print(f"\n{'='*70}")
    print(f"Training null models for {edge_type}")
    print(f"{'='*70}")
    
    # Load training data (perms 1-20)
    print(f"Loading training data from permutations {training_perm_range[0]}-{training_perm_range[1]-1}...")
    train_freq = load_null_edge_frequencies(
        edge_type, 
        range(training_perm_range[0], training_perm_range[1])
    )
    
    if len(train_freq) == 0:
        print(f"  ⚠ No training data found for {edge_type}, skipping...")
        continue
    
    print(f"  Training samples: {len(train_freq):,}")
    print(f"  Source degree range: {train_freq['source_degree'].min()}-{train_freq['source_degree'].max()}")
    print(f"  Target degree range: {train_freq['target_degree'].min()}-{train_freq['target_degree'].max()}")
    print(f"  Mean edge probability: {train_freq['edge_probability'].mean():.4f}")
    
    # Prepare features and targets
    X_train = train_freq[['source_degree', 'target_degree']].values
    y_train = train_freq['edge_probability'].values
    
    # Train Polynomial Logistic Regression
    print(f"\nTraining Polynomial Logistic Regression...")
    poly_features = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly = poly_features.fit_transform(X_train)
    
    poly_model = Ridge(alpha=1.0, random_state=42)
    poly_model.fit(X_train_poly, y_train)
    
    # Train Random Forest
    print(f"Training Random Forest...")
    rf_model = RandomForestRegressor(
        n_estimators=100, 
        max_depth=10, 
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    
    # Save models
    poly_model_file = results_dir / f'{edge_type}_poly_null.pkl'
    poly_features_file = results_dir / f'{edge_type}_poly_features.pkl'
    rf_model_file = results_dir / f'{edge_type}_rf_null.pkl'
    
    joblib.dump(poly_model, poly_model_file)
    joblib.dump(poly_features, poly_features_file)
    joblib.dump(rf_model, rf_model_file)
    
    print(f"  ✓ Saved models:")
    print(f"    - {poly_model_file.name}")
    print(f"    - {poly_features_file.name}")
    print(f"    - {rf_model_file.name}")
    
    training_results.append({
        'edge_type': edge_type,
        'n_training_samples': len(train_freq),
        'mean_edge_prob': train_freq['edge_probability'].mean(),
        'poly_model_file': str(poly_model_file),
        'rf_model_file': str(rf_model_file)
    })

training_df = pd.DataFrame(training_results)
print(f"\n{'='*70}")
print(f"TRAINING COMPLETE")
print(f"{'='*70}")
print(f"Successfully trained models for {len(training_df)} edge types")
print(f"Total model files: {len(training_df) * 3}")

## 3. Validate on Held-Out Nulls

In [ ]:
validation_results = []

for edge_type in edge_types_to_process:
    print(f"\nValidating {edge_type}...")
    
    # Check if models exist
    poly_model_file = results_dir / f'{edge_type}_poly_null.pkl'
    poly_features_file = results_dir / f'{edge_type}_poly_features.pkl'
    rf_model_file = results_dir / f'{edge_type}_rf_null.pkl'
    
    if not all([poly_model_file.exists(), rf_model_file.exists()]):
        print(f"  ⚠ Models not found, skipping...")
        continue
    
    # Load validation data (perms 21-30)
    val_freq = load_null_edge_frequencies(
        edge_type,
        range(validation_perm_range[0], validation_perm_range[1])
    )
    
    if len(val_freq) == 0:
        print(f"  ⚠ No validation data found, skipping...")
        continue
    
    X_val = val_freq[['source_degree', 'target_degree']].values
    y_val = val_freq['edge_probability'].values
    
    # Load models
    poly_model = joblib.load(poly_model_file)
    poly_features = joblib.load(poly_features_file)
    rf_model = joblib.load(rf_model_file)
    
    # Predict with Polynomial LogReg
    X_val_poly = poly_features.transform(X_val)
    y_pred_poly = poly_model.predict(X_val_poly)
    y_pred_poly = np.clip(y_pred_poly, 0, 1)  # Ensure valid probabilities
    
    # Predict with Random Forest
    y_pred_rf = rf_model.predict(X_val)
    y_pred_rf = np.clip(y_pred_rf, 0, 1)
    
    # Evaluate Polynomial LogReg
    corr_poly, _ = pearsonr(y_val, y_pred_poly)
    mae_poly = np.abs(y_val - y_pred_poly).mean()
    rmse_poly = np.sqrt(((y_val - y_pred_poly)**2).mean())
    
    # Evaluate Random Forest
    corr_rf, _ = pearsonr(y_val, y_pred_rf)
    mae_rf = np.abs(y_val - y_pred_rf).mean()
    rmse_rf = np.sqrt(((y_val - y_pred_rf)**2).mean())
    
    print(f"  Validation samples: {len(val_freq):,}")
    print(f"  Polynomial LogReg: r={corr_poly:.4f}, MAE={mae_poly:.4f}, RMSE={rmse_poly:.4f}")
    print(f"  Random Forest:     r={corr_rf:.4f}, MAE={mae_rf:.4f}, RMSE={rmse_rf:.4f}")
    
    # Error analysis by degree (using RF predictions)
    error_df = analyze_errors_by_degree(y_pred_rf, y_val, X_val[:, 0], X_val[:, 1])
    error_df['edge_type'] = edge_type
    
    # Save error analysis
    error_file = results_dir / f'{edge_type}_error_analysis.csv'
    error_df.to_csv(error_file, index=False)
    
    validation_results.append({
        'edge_type': edge_type,
        'n_validation_samples': len(val_freq),
        'poly_correlation': corr_poly,
        'poly_mae': mae_poly,
        'poly_rmse': rmse_poly,
        'rf_correlation': corr_rf,
        'rf_mae': mae_rf,
        'rf_rmse': rmse_rf
    })

validation_df = pd.DataFrame(validation_results)

print(f"\n{'='*70}")
print(f"VALIDATION COMPLETE")
print(f"{'='*70}")
print(f"\nValidation Summary:")
print(f"  Mean Poly correlation: {validation_df['poly_correlation'].mean():.4f}")
print(f"  Mean RF correlation:   {validation_df['rf_correlation'].mean():.4f}")
print(f"  Mean Poly RMSE:        {validation_df['poly_rmse'].mean():.4f}")
print(f"  Mean RF RMSE:          {validation_df['rf_rmse'].mean():.4f}")

## 4. Save Validation Results

In [ ]:
# Save validation results
validation_file = results_dir / 'validation_results.csv'
validation_df.to_csv(validation_file, index=False)
print(f"Saved validation results to: {validation_file}")

# Save training summary
training_file = results_dir / 'training_summary.csv'
training_df.to_csv(training_file, index=False)
print(f"Saved training summary to: {training_file}")

## 5. Visualize Validation Results

In [ ]:
# Plot validation performance comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Correlation comparison
ax = axes[0]
x = np.arange(len(validation_df))
width = 0.35

ax.bar(x - width/2, validation_df['poly_correlation'], width, label='Polynomial LogReg', alpha=0.7)
ax.bar(x + width/2, validation_df['rf_correlation'], width, label='Random Forest', alpha=0.7)
ax.set_xlabel('Edge Type', fontsize=12)
ax.set_ylabel('Correlation', fontsize=12)
ax.set_title('Validation Correlation by Edge Type', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(validation_df['edge_type'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.axhline(y=0.7, color='red', linestyle='--', alpha=0.5, label='Target (0.7)')

# RMSE comparison
ax = axes[1]
ax.bar(x - width/2, validation_df['poly_rmse'], width, label='Polynomial LogReg', alpha=0.7)
ax.bar(x + width/2, validation_df['rf_rmse'], width, label='Random Forest', alpha=0.7)
ax.set_xlabel('Edge Type', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Validation RMSE by Edge Type', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(validation_df['edge_type'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Scatter: Poly vs RF correlation
ax = axes[2]
ax.scatter(validation_df['poly_correlation'], validation_df['rf_correlation'], s=100, alpha=0.6)
ax.plot([0, 1], [0, 1], 'r--', alpha=0.5)
ax.set_xlabel('Polynomial LogReg Correlation', fontsize=12)
ax.set_ylabel('Random Forest Correlation', fontsize=12)
ax.set_title('Model Comparison', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

for idx, row in validation_df.iterrows():
    ax.annotate(row['edge_type'], 
               (row['poly_correlation'], row['rf_correlation']),
               fontsize=8, alpha=0.7)

plt.tight_layout()
plt.savefig(results_dir / 'validation_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved validation performance plot")

## 6. Summary Statistics

In [ ]:
print("\n" + "="*70)
print("NULL MODEL TRAINING SUMMARY")
print("="*70)

print(f"\nTraining:")
print(f"  Permutations: {training_perm_range[0]}-{training_perm_range[1]-1}")
print(f"  Edge types: {len(training_df)}")
print(f"  Models created: {len(training_df) * 2} (Poly + RF)")

print(f"\nValidation:")
print(f"  Permutations: {validation_perm_range[0]}-{validation_perm_range[1]-1}")
print(f"  Edge types: {len(validation_df)}")

print(f"\nPerformance (held-out null networks):")
print(f"  Polynomial LogReg:")
print(f"    Mean correlation: {validation_df['poly_correlation'].mean():.4f} (±{validation_df['poly_correlation'].std():.4f})")
print(f"    Mean RMSE:        {validation_df['poly_rmse'].mean():.4f} (±{validation_df['poly_rmse'].std():.4f})")
print(f"  Random Forest:")
print(f"    Mean correlation: {validation_df['rf_correlation'].mean():.4f} (±{validation_df['rf_correlation'].std():.4f})")
print(f"    Mean RMSE:        {validation_df['rf_rmse'].mean():.4f} (±{validation_df['rf_rmse'].std():.4f})")

print(f"\nBest performing edge types (by RF correlation):")
top_5 = validation_df.nlargest(5, 'rf_correlation')[['edge_type', 'rf_correlation', 'rf_rmse']]
for idx, row in top_5.iterrows():
    print(f"  {row['edge_type']}: r={row['rf_correlation']:.4f}, RMSE={row['rf_rmse']:.4f}")

print(f"\nOutput files:")
print(f"  Model files: {len(list(results_dir.glob('*_null.pkl')))} + {len(list(results_dir.glob('*_features.pkl')))}")
print(f"  Validation results: {validation_file.name}")
print(f"  Training summary: {training_file.name}")
print(f"  Error analysis files: {len(list(results_dir.glob('*_error_analysis.csv')))}")

print("\n" + "="*70)
print("COMPLETE!")
print("="*70)